In [24]:
from crampon.core.models.massbalance import BraithwaiteModel
from crampon import cfg
import numpy as np
import geopandas as gpd
from oggm.utils import get_demo_file
from crampon.utils import GlacierDirectory
import shapely.geometry as shpg
import salem
from collections import OrderedDict
from crampon.tasks import process_custom_climate_data
from crampon import workflow
from crampon.cfg import SEC_IN_DAY
import xarray as xr
import pandas as pd

%matplotlib inline

cfg.initialize('c:\\users\\johannes\\documents\\crampon\\sandbox\\CH_params.cfg')

2018-04-26 18:29:43: oggm.cfg: Parameter file: c:\users\johannes\documents\oggm\oggm\params.cfg
2018-04-26 18:29:43: crampon.cfg: Parameter file: c:\users\johannes\documents\crampon\sandbox\CH_params.cfg


In [25]:
## A copy of OGGM's idealized_gdir with some changes, e.g. a "name" kw
def idealized_gdir(surface_h, widths_m, map_dx, flowline_dx=1, name=None, identifier=None, coords=None,
                   base_dir=None, reset=False):
    """Creates a glacier directory with flowline input data only.
    This is useful for testing, or for idealized experiments.
    Parameters
    ----------
    surface_h : ndarray
        the surface elevation of the flowline's grid points (in m).
    widths_m : ndarray
        the widths of the flowline's grid points (in m).
    map_dx : float
        the grid spacing (in m)
    flowline_dx : int
        the flowline grid spacing (in units of map_dx, often it should be 1)
    base_dir : str
        path to the directory where to open the directory.
        Defaults to `cfg.PATHS['working_dir'] + /per_glacier/`
    reset : bool, default=False
        empties the directory at construction
    Returns
    -------
    a GlacierDirectory instance
    """

    from oggm.core.centerlines import Centerline

    # Area from geometry
    area_km2 = np.sum(widths_m * map_dx * flowline_dx) * 1e-6

    # Dummy entity - should probably also change the geometry
    entity = gpd.read_file(get_demo_file('Hintereisferner_RGI5.shp')).iloc[0]
    entity.Area = area_km2
    if coords:
        entity.CenLat = coords[0]
        entity.CenLon = coords[1]
    else:
        entity.CenLat = 0
        entity.CenLon = 0
    if name:
        entity.Name = name
    else:
        entity.Name = ''
    if identifier:
        entity.RGIId = identifier
    else:
        entity.RGIId = 'RGI50-00.00000'
    entity.O1Region = '00'
    entity.O2Region = '0'
    gdir = GlacierDirectory(entity, base_dir=base_dir, reset=reset)

    # Idealized flowline
    if coords:
        coords = np.array(coords)
    else:
        coords = np.arange(0, len(surface_h)-0.5, 1)
    line = shpg.LineString(np.vstack([coords, coords * 0.]).T)
    fl = Centerline(line, dx=flowline_dx, surface_h=surface_h)
    fl.widths = widths_m / map_dx
    fl.is_rectangular = np.ones(fl.nx).astype(np.bool)
    gdir.write_pickle([fl], 'inversion_flowlines')

    # Idealized map
    grid = salem.Grid(nxny=(1, 1), dxdy=(map_dx, map_dx), x0y0=(0, 0))
    grid.to_json(gdir.get_filepath('glacier_grid'))

    return gdir

#### Set some params

In [26]:
cfg.PATHS['working_dir'] = 'c:\\users\\johannes\\documents\\modelruns\\idealized'

#### Heights of the cores

In [27]:
# cg= Colle Gnifetti, fi=Fiescherhorn, pz=Piz Zupo, gr=Grenzgletscher

h_dict = OrderedDict({'schwikowski_cg': ('RGI50-00.00000', 4455, 45.93055556, 7.87583333, 0.46, 0.08), 
                      'schwikowski_fi': ('RGI50-00.00001', 3900, 46.55088889, 8.06694444, 1.4, 0.), 
                      'schwikowski_pz': ('RGI50-00.00002', 3850, 46.36703, 9.92824, 2.6, 0.8), 
                      'schwikowski_gr': ('RGI50-00.00003', 4200, 45.92444444,7.86750000, 0.3, 0.)})

columns=['id', 'height', 'lat', 'lon', 'mean_acc', 'mean_acc_unc']
data = pd.DataFrame.from_dict(h_dict, orient='index')
data.columns = columns

In [28]:
data

,id,height,lat,lon,mean_acc,mean_acc_unc
schwikowski_gr,RGI50-00.00003,4200,45.924444,7.867500,0.30,0.00
schwikowski_fi,RGI50-00.00001,3900,46.550889,8.066944,1.40,0.00
schwikowski_cg,RGI50-00.00000,4455,45.930556,7.875833,0.46,0.08
schwikowski_pz,RGI50-00.00002,3850,46.367030,9.928240,2.60,0.80


In [29]:
def fit_mb_to_mean_acc(gdir):
    old_mb = gdir.read_pickle('mb_daily')
    mean_mb = old_mb.apply(np.nanmean)
    
    mean_acc = data.loc[data.id == gdir.rgi_id].mean_acc.values[0] /365.25
    
    factor = mean_mb.apply(lambda x: x / mean_acc).MB.values
    print(mean_mb, mean_acc, factor)
    new_mb = old_mb.apply(lambda x: x / factor)
    
    return new_mb

In [30]:
mb_model = BraithwaiteModel

for i, row in data.iterrows():
    gdir = idealized_gdir(np.array([row.height]), np.ndarray([1.]), map_dx=1., identifier=row.id, 
                          coords=(row.lat, row.lon))
    
    process_custom_climate_data(gdir)
    
    # Start with any params and then trim on accum rate later
    mb_model = BraithwaiteModel(gdir, mu_ice=10., mu_snow=5., prcp_fac=1.0, bias=0.)
    
    mb = []
    for date in mb_model.tspan_meteo:
        
        if date.month == 1 and date.day == 1:
            print(date)
        # Get the mass balance and convert to m per day
        tmp = mb_model.get_daily_mb(np.array([row.height]), date=date) * SEC_IN_DAY * cfg.RHO / 1000.
        mb.append(tmp)
       
    mb_ds = xr.Dataset({'MB': (['time', 'n'],np.array(mb))},
                           coords={'time': pd.to_datetime(mb_model.time_elapsed),
                                  'n': (['n'], [1])},
                           attrs={'id': gdir.rgi_id,
                                  'name': gdir.name})
    gdir.write_pickle(mb_ds, 'mb_daily')
    
    new_mb = fit_mb_to_mean_acc(gdir)
    
    gdir.write_pickle(new_mb, 'mb_daily_rescaled')

C:\Program Files\Anaconda3\lib\site-packages\ipykernel\__main__.py:2: VisibleDeprecationWarning: using a non-integer number instead of an integer will result in an error in the future
  from ipykernel import kernelapp as app
2018-04-26 18:29:44: crampon.core.preprocessing.climate: (RGI50-00.00003) process_custom_climate_data_crampon


1961-01-01 00:00:00
1962-01-01 00:00:00
1963-01-01 00:00:00
1964-01-01 00:00:00
1965-01-01 00:00:00
1966-01-01 00:00:00
1967-01-01 00:00:00
1968-01-01 00:00:00
1969-01-01 00:00:00
1970-01-01 00:00:00
1971-01-01 00:00:00
1972-01-01 00:00:00
1973-01-01 00:00:00
1974-01-01 00:00:00
1975-01-01 00:00:00
1976-01-01 00:00:00
1977-01-01 00:00:00
1978-01-01 00:00:00
1979-01-01 00:00:00
1980-01-01 00:00:00
1981-01-01 00:00:00
1982-01-01 00:00:00
1983-01-01 00:00:00
1984-01-01 00:00:00
1985-01-01 00:00:00
1986-01-01 00:00:00
1987-01-01 00:00:00
1988-01-01 00:00:00
1989-01-01 00:00:00
1990-01-01 00:00:00
1991-01-01 00:00:00
1992-01-01 00:00:00
1993-01-01 00:00:00
1994-01-01 00:00:00
1995-01-01 00:00:00
1996-01-01 00:00:00
1997-01-01 00:00:00
1998-01-01 00:00:00
1999-01-01 00:00:00
2000-01-01 00:00:00
2001-01-01 00:00:00
2002-01-01 00:00:00
2003-01-01 00:00:00
2004-01-01 00:00:00
2005-01-01 00:00:00
2006-01-01 00:00:00
2007-01-01 00:00:00
2008-01-01 00:00:00
2009-01-01 00:00:00
2010-01-01 00:00:00


C:\Program Files\Anaconda3\lib\site-packages\ipykernel\__main__.py:2: VisibleDeprecationWarning: using a non-integer number instead of an integer will result in an error in the future
  from ipykernel import kernelapp as app
2018-04-26 18:31:14: crampon.core.preprocessing.climate: (RGI50-00.00001) process_custom_climate_data_crampon


<xarray.Dataset>
Dimensions:  ()
Data variables:
    MB       float64 0.007273 0.00082135523614 8.85440097551247
1961-01-01 00:00:00
1962-01-01 00:00:00
1963-01-01 00:00:00
1964-01-01 00:00:00
1965-01-01 00:00:00
1966-01-01 00:00:00
1967-01-01 00:00:00
1968-01-01 00:00:00
1969-01-01 00:00:00
1970-01-01 00:00:00
1971-01-01 00:00:00
1972-01-01 00:00:00
1973-01-01 00:00:00
1974-01-01 00:00:00
1975-01-01 00:00:00
1976-01-01 00:00:00
1977-01-01 00:00:00
1978-01-01 00:00:00
1979-01-01 00:00:00
1980-01-01 00:00:00
1981-01-01 00:00:00
1982-01-01 00:00:00
1983-01-01 00:00:00
1984-01-01 00:00:00
1985-01-01 00:00:00
1986-01-01 00:00:00
1987-01-01 00:00:00
1988-01-01 00:00:00
1989-01-01 00:00:00
1990-01-01 00:00:00
1991-01-01 00:00:00
1992-01-01 00:00:00
1993-01-01 00:00:00
1994-01-01 00:00:00
1995-01-01 00:00:00
1996-01-01 00:00:00
1997-01-01 00:00:00
1998-01-01 00:00:00
1999-01-01 00:00:00
2000-01-01 00:00:00
2001-01-01 00:00:00
2002-01-01 00:00:00
2003-01-01 00:00:00
2004-01-01 00:00:00
2005-01

C:\Program Files\Anaconda3\lib\site-packages\ipykernel\__main__.py:2: VisibleDeprecationWarning: using a non-integer number instead of an integer will result in an error in the future
  from ipykernel import kernelapp as app
2018-04-26 18:32:41: crampon.core.preprocessing.climate: (RGI50-00.00000) process_custom_climate_data_crampon


<xarray.Dataset>
Dimensions:  ()
Data variables:
    MB       float64 0.007912 0.00383299110198 2.064076879681488
1961-01-01 00:00:00
1962-01-01 00:00:00
1963-01-01 00:00:00
1964-01-01 00:00:00
1965-01-01 00:00:00
1966-01-01 00:00:00
1967-01-01 00:00:00
1968-01-01 00:00:00
1969-01-01 00:00:00
1970-01-01 00:00:00
1971-01-01 00:00:00
1972-01-01 00:00:00
1973-01-01 00:00:00
1974-01-01 00:00:00
1975-01-01 00:00:00
1976-01-01 00:00:00
1977-01-01 00:00:00
1978-01-01 00:00:00
1979-01-01 00:00:00
1980-01-01 00:00:00
1981-01-01 00:00:00
1982-01-01 00:00:00
1983-01-01 00:00:00
1984-01-01 00:00:00
1985-01-01 00:00:00
1986-01-01 00:00:00
1987-01-01 00:00:00
1988-01-01 00:00:00
1989-01-01 00:00:00
1990-01-01 00:00:00
1991-01-01 00:00:00
1992-01-01 00:00:00
1993-01-01 00:00:00
1994-01-01 00:00:00
1995-01-01 00:00:00
1996-01-01 00:00:00
1997-01-01 00:00:00
1998-01-01 00:00:00
1999-01-01 00:00:00
2000-01-01 00:00:00
2001-01-01 00:00:00
2002-01-01 00:00:00
2003-01-01 00:00:00
2004-01-01 00:00:00
2005-0

C:\Program Files\Anaconda3\lib\site-packages\ipykernel\__main__.py:2: VisibleDeprecationWarning: using a non-integer number instead of an integer will result in an error in the future
  from ipykernel import kernelapp as app
2018-04-26 18:34:08: crampon.core.preprocessing.climate: (RGI50-00.00002) process_custom_climate_data_crampon


<xarray.Dataset>
Dimensions:  ()
Data variables:
    MB       float64 0.006525 0.00125941136208 5.181002718777869
1961-01-01 00:00:00
1962-01-01 00:00:00
1963-01-01 00:00:00
1964-01-01 00:00:00
1965-01-01 00:00:00
1966-01-01 00:00:00
1967-01-01 00:00:00
1968-01-01 00:00:00
1969-01-01 00:00:00
1970-01-01 00:00:00
1971-01-01 00:00:00
1972-01-01 00:00:00
1973-01-01 00:00:00
1974-01-01 00:00:00
1975-01-01 00:00:00
1976-01-01 00:00:00
1977-01-01 00:00:00
1978-01-01 00:00:00
1979-01-01 00:00:00
1980-01-01 00:00:00
1981-01-01 00:00:00
1982-01-01 00:00:00
1983-01-01 00:00:00
1984-01-01 00:00:00
1985-01-01 00:00:00
1986-01-01 00:00:00
1987-01-01 00:00:00
1988-01-01 00:00:00
1989-01-01 00:00:00
1990-01-01 00:00:00
1991-01-01 00:00:00
1992-01-01 00:00:00
1993-01-01 00:00:00
1994-01-01 00:00:00
1995-01-01 00:00:00
1996-01-01 00:00:00
1997-01-01 00:00:00
1998-01-01 00:00:00
1999-01-01 00:00:00
2000-01-01 00:00:00
2001-01-01 00:00:00
2002-01-01 00:00:00
2003-01-01 00:00:00
2004-01-01 00:00:00
2005-0